In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import stackstac
import planetary_computer as pc
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# Import modular function
from env_function import environmental_variables

# Start Dask (safe + required)
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="8GB"
)
client = Client(cluster)

client

In [ ]:
bbox = (-82.7167, 27.5833, -82.3833, 28.0333)  # Tampa Bay
start_date = "2019-01-01"
end_date = "2024-12-31"

env_data = environmental_variables(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    variables=["sst", "precip"]
)

sst_lazy = env_data["sst"]          # xarray DataArray (lazy)
precip_items = env_data["precip"]   # list of STAC Items

type(sst_lazy), type(precip_items)


In [ ]:
import pandas as pd

# Refresh SAS tokens (REQUIRED)
for item in precip_items:
    pc.sign_inplace(item)

# Make start/end timezone-aware (UTC)
start = pd.Timestamp("2019-01-01", tz="UTC")
end = pd.Timestamp("2024-12-31", tz="UTC")

# Filter items strictly to 2019–2024
precip_filtered = [
    item for item in precip_items
    if start <= pd.to_datetime(item.datetime) <= end
]

len(precip_filtered)


In [ ]:
# Stack STAC items (NO chunks here)
precip_stack = stackstac.stack(
    precip_filtered,
    assets=["cog"],
    epsg=4326,
    fill_value=np.nan
)

# Apply chunking AFTER stacking (this is the correct pattern)
precip_stack = precip_stack.chunk({"time": 30})

# Monthly aggregation
precip_monthly = (
    precip_stack
    .mean(dim=["x", "y"])          # spatial mean
    .squeeze(drop=True)
    .resample(time="1ME")          # monthly
    .sum(min_count=1)              # precipitation sum
)


In [ ]:
final_ds = xr.Dataset({
    "sst": sst_lazy,
    "precip": precip_monthly
})

final_df = final_ds.compute().to_dataframe()
final_df.head()


In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(final_df.index, final_df["sst"], label="Monthly SST (°C)")
plt.plot(final_df.index, final_df["precip"], label="Monthly Precip (mm)")
plt.title("Monthly SST & Precipitation (2019–2024)")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- 1. Separate DataFrames ---

# Select the SST column and save it to a new DataFrame
sst_df = final_df[['sst']].copy()

# Select the Precipitation column and save it to a new DataFrame
precip_df = final_df[['precip']].copy()

# --- 2. Define File Paths ---
sst_path = "kalu_sst_data.csv"
precip_path = "kalu_precip_data.csv"

# --- 3. Save to CSV ---
sst_df.to_csv(sst_path, index=True, index_label='time')
print(f"✔ Successfully saved SST data to: {sst_path}")

precip_df.to_csv(precip_path, index=True, index_label='time')
print(f"✔ Successfully saved Precipitation data to: {precip_path}")

# --- 4. Cleanup Dask Resources (Crucial) ---
try:
    client.close()
    cluster.close()
    print("\nDask client and cluster successfully shut down.")
except Exception as e:
    print(f"Could not shut down Dask resources: {e}")

# Display the paths

In [ ]:
from kalu_analysis import (
    load_env_timeseries,
    join_indices_and_env,
    compute_correlations_and_rmse,
)
from grace_functions import compute_indices

# 1) Define study box + time
bbox = (-82.7167, 27.5833, -82.3833, 28.0333)  # Tampa Bay
start_date = "2019-01-01"
end_date = "2024-12-31"

# 2) Grace's function: monthly NDWI/NDTI/NDCI over Tampa Bay
indices_ds = compute_indices(bbox=bbox, start_date=start_date, end_date=end_date)

# 3) Kalu's function: monthly SST
sst_da = load_env_timeseries(bbox=bbox, start_date=start_date, end_date=end_date)

# 4) Join and compute stats
df = join_indices_and_env(indices_ds, sst_da)
stats = compute_correlations_and_rmse(df)

stats

